In [8]:
import scipy.io
import numpy as np
import os
join = os.path.join

from PIL import Image


import rectify
import cv06, cv07
import tools

In [17]:
root_dir = '/home/hartvi/zs22/TDV/scene_1/'
img_dir = root_dir + 'images/'
zsdir = '/home/hartvi/zs22/'
Xs = np.load(zsdir+'allX.npy')
Ps = np.load(zsdir+"Ps.npy")
K = cv06.K
invK = cv06.invK


def squarifyP(P):
    return np.vstack([P, [0,0,0,1]])

def rel_P(P1, P2):
    return (squarifyP(P2) @ np.linalg.inv(squarifyP(P1)))[:3, :]


task = [] # task variable for all data

for i in range(1, 13):
    for k in range(i+1, 13):
        imga = '{0:02}'.format(i)
        im_a = Image.open(join(img_dir, imga + '.jpg'))
        imgb = '{0:02}'.format(k)
        im_b = Image.open(join(img_dir, imgb + '.jpg'))
        Rt_ab = rel_P(Ps[i-1], Ps[k-1])
        F_ab = tools.calc_F(invK=invK, R=Rt_ab[:3, :3], t=Rt_ab[:, -1])

        u_a = tools.e2p(np.loadtxt(join(root_dir, "corresp", "u_{:02d}.txt".format(i))).T)
        u_b = tools.e2p(np.loadtxt(join(root_dir, "corresp", "u_{:02d}.txt".format(k))).T)
        corresp = np.loadtxt(
            join(root_dir, "corresp", "m_{:02d}_{:02d}.txt".format(i, k)), 
            dtype=int
        ).T
        u_a_p = u_a[:, corresp[0]]
        u_b_p = u_b[:, corresp[1]]
        inlier_indices = cv07.eval_inliers(u_a_p, u_b_p, F_ab, 1.0)
        u_a_i = u_a_p[:, inlier_indices]
        u_b_i = u_b_p[:, inlier_indices]
        
        im_a_arr = np.array(im_a)
        im_b_arr = np.array(im_b)
        [H_a, H_b, im_a_r, im_b_r] = rectify.rectify( F_ab, im_a_arr, im_b_arr )
        u_a_r = H_a @ u_a_i
        u_b_r = H_b @ u_b_i
        
        seeds = np.vstack( ( u_a_r[0,:], u_b_r[0], ( u_a_r[1] + u_b_r[1] ) / 2 ) ).T
        task_i = np.array( [ im_a_r, im_b_r, seeds ], dtype=object )
        task += [ task_i ]

In [20]:
print(task)
np.save("task.npy", np.array(task))

[array([array([[0, 0, 0, ..., 0, 0, 0],
              [0, 0, 0, ..., 0, 0, 0],
              [0, 0, 0, ..., 0, 0, 0],
              ...,
              [0, 0, 0, ..., 0, 0, 0],
              [0, 0, 0, ..., 0, 0, 0],
              [0, 0, 0, ..., 0, 0, 0]], dtype=uint8),
       array([[  0,   0,   0, ...,   0,   0,   0],
              [  0,   0,   0, ...,   0,   0,   0],
              [  0,   0,   0, ...,   0,   0,   0],
              ...,
              [  0,   0,   0, ..., 176, 176,   0],
              [  0,   0,   0, ..., 176, 176,  18],
              [  0,   0,   0, ...,  12,  12,   0]], dtype=uint8),
       array([[  90.93224067,  711.21516628, 1680.13620228],
              [ 108.09800664,   30.25247932, 1272.13876796],
              [ 116.22759672,   35.46220672, 1558.57867467],
              ...,
              [ 680.81156197, 1398.83484774, 1112.89151906],
              [ 683.4446192 , 1400.47575064, 1115.05935691],
              [ 849.41448401, 1612.90230224,  840.06104625]])],
   